In [ ]:
!pip install langchain_community langchain-openai langchainhub chromadb langchain pypdf langchain_postgres psycopg2-binary python-docx pymysql 'crewai[tools]'==0.32.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
import os
from docx import Document
from langchain import OpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import initialize_agent, Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''


def load_local_doc(file_path: str) -> str:
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            return f.read()
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format. Use .txt or .docx")

# Load the document
file_path = 'Engine overheat scenarios.docx'  # Update with your file path
document_content = load_local_doc(file_path)

# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.create_documents([document_content])

# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings()

# Store embeddings in ChromaDB
vector_store = Chroma.from_documents(docs, embeddings)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
retrieval_qa_chat_prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [ ]:
import sqlite3
conn = sqlite3.connect('demo.db')
cursor = conn.cursor()

with open('machine_metrics.sql', 'r') as sql_file:
    sql_script = sql_file.read()

cursor.executescript(sql_script)
conn.commit()
cursor.execute('SELECT * FROM machine_running_metrics')
rows = cursor.fetchall()

for row in rows:
    print(row)
conn.close()

(1400.0, 110.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd123')
(1000.0, 250.0, 100.0, 250.0, '2024-09-29 00:00:00', 'abcd124')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from langchain_community.agent_toolkits import create_sql_agent
import urllib.parse
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from textwrap import dedent

sqlite_uri = "sqlite:////content/demo.db"  # Ensure the correct path to your .db file

# Create the database object for SQLite
db = SQLDatabase.from_uri(sqlite_uri)

@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

# print(tables_schema.run("salaries"))

@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

# execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

# check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")


Using Tool: list_tables


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def load_local_doc(file_path: str) -> str:
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            return f.read()
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format. Use .txt or .docx")

# Load the document
file_path = 'Engine overheat scenarios.docx'  # Update with your file path
document_content = load_local_doc(file_path)

# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.create_documents([document_content])

# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings()

# Store embeddings in ChromaDB
vector_store = Chroma.from_documents(docs, embeddings)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
retrieval_qa_chat_prompt = hub.pull("rlm/rag-prompt")

# state = {}
def simple_preprocessing(llm, new_question):
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    docs = retriever.get_relevant_documents(new_question)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    context = format_docs(docs)

    output_parser = StrOutputParser()

    chain = retrieval_qa_chat_prompt | llm | output_parser

    return chain.invoke({"context": context, "question": new_question})

@tool("handle_query")
def handle_query(input_query: str) -> str:
    """
    Handle queries related to problem being faced by machine give the metrics for the problem mentioned.
    just give high level cases metrics needs to be check and what its value should be, give response point wise
    """
    query = ("give me the metrics for the problem mentioned. \n"
         "just give high level cases metrics needs to be check and what its value should be \n"
         "\n problem: ") + input_query
    response = simple_preprocessing(llm, query)
    return response

@tool("get_metrics")
def get_metrics(handle_query_response: str) -> str:
    """
    Given the reult from the query handler give point wise metrics to be sent to a SQL developer
    """
    metrics_to_check = llm.invoke("give the point wise metrics to be sent to a SQL agents to query the database \n " + handle_query_response)
    return metrics_to_check.content

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [ ]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal=" Construct and execute SQL queries based on the data received from Knowledge Agent",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

knowledge_agent = Agent(
    role="Knowledge Agent",
    goal="You provide the point wise metrics to be sent to a Database Developer based on the problem mentioned in request",
    backstory=dedent(
        """
        You are a knowledge agent who is master at providing the point wise
        metrics to be sent to a Database Developer based on the problem mentioned in request.
    """
    ),
    llm=llm,
    tools=[handle_query, get_metrics],
    allow_delegation=False,
)

analysing_agent = Agent(
    role="Analysis Expert",
    goal="you get the metrics from the Knowledge Agent and response from the Database Developer to infer the problem and give its diagnosis.",
    backstory=dedent(
        """
        Figure out what can be the root cause for the problem. You get knowledge from the handle_query and result from
        database developer to figure out the problem and give its diagnosis.
    """
    ),
    llm=llm,
    tools=[handle_query],
    allow_delegation=False,
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
extract_data = Task(
    description="Extract the metrics need to be checked by Database developer for the given machine problem: {query}.",
    expected_output="point wise metrics to be sent to a Database Developer",
    agent=knowledge_agent
)
query_data = Task(
    description="Get the pointwise metrics from knowledge agent and query database to give your findings on the metrics",
    expected_output="give a combined text of which what metrics got analysed and what was the finding for a particulat machine",
    agent=sql_dev,
    context=[extract_data]
)
diagnosis_problem = Task(
    description=dedent(
        """
        figuring out what can be the root cause for the problem: {query} and give a diagnosis of the problem.
    """
    ),
    expected_output="diagnosis report",
    agent=analysing_agent,
    context=[query_data],
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
crew = Crew(
    agents=[sql_dev, knowledge_agent, analysing_agent],
    tasks=[extract_data, query_data, diagnosis_problem],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="crew.log",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
inputs = {
    "query": "engine of machine abc123 is overheating"
}

result = crew.kickoff(inputs=inputs)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-pa

 [2024-10-01 01:51:24][DEBUG]: == Working Agent: Knowledge Agent
 [2024-10-01 01:51:24][INFO]: == Starting Task: Extract the metrics need to be checked by Database developer for the given machine problem: engine of machine abc123 is overheating.
 

The high-level metrics to check for the engine overheating problem are Exhaust Gas Temperature (EGT) and Coolant Temperature. The values to be monitored are EGT > 1,300°F and Coolant Temperature > 220°F. These metrics indicate potential issues with air-fuel mixture, turbocharger, radiator, or coolant circulation.



/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(


 

1. Exhaust Gas Temperature (EGT) - This metric should be greater than 1,300°F.
2. Coolant Temperature - This metric should be greater than 220°F.



/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(


...

 [2024-10-01 01:51:34][DEBUG]: == [Senior Database Developer] Task output: Point wise metrics to be sent to a Database Developer for the engine overheating problem of machine abc123:
1. Exhaust Gas Temperature (EGT) should be greater than 1,300°F.
2. Coolant Temperature should be greater than 220°F.

Based on the data from the `machine_running_metrics` table, the metrics for machine abc123 are as follows:
- Exhaust Gas Temperature (EGT): 1400.0°F
- Coolant Temperature: 110.0°F

Therefore, the findings for machine abc123 are:
1. The Exhaust Gas Temperature (EGT) is greater than 1,300°F, meeting the criteria.
2. The Coolant Temperature is less than 220°F, not meeting the criteria.

Further analysis and action may be required to address the coolant temperature issue for machine abc123.


 [2024-10-01 01:51:34][DEBUG]: == Working Agent: Analysis Expert
 [2024-10-01 01:51:34][INFO]: == Starting Task: 
figuring out what can be the root cause for the problem: engine of machine abc123 is overh

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedS

 

The high-level metrics for the engine overheating problem of machine abc123 include Exhaust Gas Temperature (EGT) exceeding 1,300°F and Coolant Temperature surpassing 220°F. The ideal values for these metrics should be below the specified thresholds to prevent engine overheating.



/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(


 [2024-10-01 01:51:38][DEBUG]: == [Analysis Expert] Task output: Diagnosis Report:
Based on the analysis of the metrics for machine abc123, it is evident that the Exhaust Gas Temperature (EGT) is within the acceptable range, meeting the criteria. However, the Coolant Temperature is below the required threshold of 220°F, indicating a potential issue with the cooling system. Further investigation and action are recommended to address the coolant temperature problem and prevent engine overheating in machine abc123.




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
